
# Earth Engine Companion

<a target='_blank' href='https://colab.research.google.com/github/google/earthengine-community/blob/master/experimental/functionsmith/ee_companion.ipynb'>   <img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/> </a>

**USING THIS AGENT IS UNSAFE**. It directly runs LLM-produced code, and thus
should only be used for demonstration purposes. However, Colab serves as
a moderately effective sandbox - the damage would be limited to whatever
this notebook has access to.

## Configuration

To run with the sample task (see `task` variable below),
[obtain a Gemini API key](https://ai.google.dev/gemini-api/docs/api-key)
and save it into a [Colab secret](https://colab.sandbox.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) named "GOOGLE_API_KEY".

Run the notebook, then scroll to the end. You will see an empty text area
and the task definition under it. Hit Enter in the task defintion input box
to start processing.

By default, the Gemini API is used. You can switch to OpenAI, Anthropic, or DeepSeek
APIs by uncommenting the relevant LLM class in the last cell. You will also
need to save ANTHROPIC_API_KEY, OPENAI_API_KEY, or DEEPSEEK_API_KEY secrets.

## Related work

A similar non-Earth-Engine-specific notebook is available [here](https://github.com/google/earthengine-community/blob/master/experimental/functionsmith/functionsmith.ipynb).

## Attribution

EE Companion and the functionsmith package were written by Simon Ilyushchenko (simonf@google.com). I am grateful to Renee Johnston and other Googlers for implementation advice, as well as to Earth Engine expert advisors Jeffrey Cardille, Erin Trochim, Morgan Crowley, and Samapriya Roy, who helped me choose the right training tasks.

In [ ]:
!pip install functionsmith

# Imports

In [ ]:
import asyncio
import copy
import enum
import inspect
import logging
import os
import sys
import time
from typing import Callable

import google.colab
from google.colab import userdata
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript

from functionsmith import code_parser
from functionsmith import executor
from functionsmith import llm


In [ ]:
import ee
import geemap
ee.Authenticate(scopes=['https://www.googleapis.com/auth/earthengine.readonly'])
ee.Initialize(project=userdata.get('EE_PROJECT_ID'))

Map = geemap.Map()
Map.add_basemap("Esri.WorldImagery", False)
Map.add("layer_manager")

from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket('earthengine-stac')

# Hardcode the name of a dataset from the community catalog used in one
# of the sample tasks.
hydrolakes_dataset_id = 'projects/sat-io/open-datasets/HydroLakes/lake_poly_v10'

In [ ]:
def get_dataset_description(dataset_id: str) -> str:
  """Fetches JSON STAC description for the given Earth Engine dataset id."""
  if not dataset_id:
    return 'ERROR: please provide dataset_id'
  with io_manager._output_area:
    print(f'\nLOOKING UP {dataset_id}')

  if dataset_id == hydrolakes_dataset_id:
    return hydrolakes_schema

  parent = dataset_id.split('/')[0]

  # Get the blob (file)
  path = os.path.join('catalog', parent, dataset_id.replace('/', '_')) + '.json'
  blob = bucket.blob(path)

  if not blob.exists():
    with io_manager._output_area:
      print(f'LOOKUP FAILED - NO SUCH DATASET\n{STARS}')
    return 'No such dataset'

  file_contents = blob.download_as_string().decode()
  with io_manager._output_area:
    print(f'LOOKUP SUCCESSFUL\n{STARS}')
  return file_contents

# System prompt

In [ ]:
system_instruction="""
You are an AI assistant designed to solve geospatial problems using Google Earth
Engine and Python. **Crucially, you must operate in an evolutionary,
step-by-step manner.**

**Your core directives are:**

1.  **Decomposition:** Analyze the user's request and **break it down into the
smallest possible logical sub-tasks.**

2.  **Incremental Execution:** Address **only one sub-task per turn.**
Propose the next step only after the current one is successfully completed
and verified.

3.  **Minimal Code:** For each step, write the **absolute minimum amount of
Python code** required to achieve that single sub-task.

4.  **Verification:** **Explicitly verify the result of each step** before
moving on. This might involve printing intermediate values, checking counts
(`.size().getInfo()`), examining properties (`.getInfo()`), or adding temporary
layers to the map for visual confirmation.

5.  **Explanation:** **Clearly state the goal of the current step** before
presenting the code. After execution, briefly explain the result and state
the goal for the *next* step.

6.  **No Premature Solutions:** **Do not provide code that attempts to solve
multiple steps or the entire problem at once.**

7.  **Function persistence:** **If you define a function in a response,
I will save it locally. Later code that you generate can refer to these
functions without redefining them.**

8.  **No variable persistence:** Variables are **not persistent**
between code executions, however. Reload data or recalculate intermediate results
as needed in each step.

**Example Interaction Flow:**

*   User provides a task.
*   You state the first small step (e.g., "First, I need to find and examine
the relevant dataset. I will fetch its description.").
*   You provide *only* the code for that step (e.g., `print(get_dataset_description(...))`).
*   User executes the code.
*   You analyze the result, state the next small step (e.g., "Okay, the dataset
looks suitable. Now I will load it as a FeatureCollection."), provide the minimal
code for *that* step, and so on.

Adhere strictly to this incremental process.

Critical: do not try to perform all steps in a single answer. Return code in
small pieces, and I will ask you to continue in the next chat message.
"""

# Task

In [ ]:
task = ''

if False:
  task = 'Show me something interesting, unexpected, and uplifting. Explain why you chose what you chose.'

if False:
  task = 'Zoom in to a random midsize city and display a recent Sentinel-2 monthly mosaic with good visualization. Do not use filterBounds()'

if False:
  task = "Compute the area of Belgium by querying the WM/geoLab/geoBoundaries/600/ADM0 dataset in Earth Engine. Show the boundaries on geemap and zoom in on them."

# The schema for the Hydrolakes dataset, which is not present in the main EE catalog.
hydrolakes_schema = """
Hylak_id	Unique lake identifier. Values range from 1 to 1,427,688.
Lake_name	Name of lake or reservoir. This field is currently only populated for lakes with an area of at least 500 km2; for large reservoirs where a name was available in the GRanD database; and for smaller lakes where a name was available in the GLWD database.
Country	Country that the lake (or reservoir) is located in. International or transboundary lakes are assigned to the country in which its corresponding lake pour point is located and may be arbitrary for pour points that fall on country boundaries.
Continent	Continent that the lake (or reservoir) is located in. Geographic continent: Africa, Asia, Europe, North America, South America, or Oceania (Australia and Pacific Islands)
Poly_src	Source of original lake polygon: CanVec; SWBD; MODIS; NHD; ECRINS; GLWD; GRanD; or Other More information on these data sources can be found in Table 1.
Lake_type	Indicator for lake type: 1: Lake 2: Reservoir 3: Lake control (i.e. natural lake with regulation structure) Note that the default value for all water bodies is 1, and only those water bodies explicitly identified as other types (mostly based on information from the GRanD database) have other values; hence the type ‘Lake’ also includes all unidentified smaller human-made reservoirs and regulated lakes.
Grand_id	ID of the corresponding reservoir in the GRanD database, or value 0 for no corresponding GRanD record. This field can be used to join additional attributes from the GRanD database.
Lake_area	Lake surface area (i.e. polygon area), in square kilometers.
Shore_len	Length of shoreline (i.e. polygon outline), in kilometers.
Shore_dev	Shoreline development, measured as the ratio between shoreline length and the circumference of a circle with the same area. A lake with the shape of a perfect circle has a shoreline development of 1, while higher values indicate increasing shoreline complexity.
Vol_total	Total lake or reservoir volume, in million cubic meters (1 mcm = 0.001 km3). For most polygons, this value represents the total lake volume as estimated using the geostatistical modeling approach by Messager et al. (2016). However, where either a reported lake volume (for lakes ≥ 500 km2) or a reported reservoir volume (from GRanD database) existed, the total volume represents this reported value. In cases of regulated lakes, the total volume represents the larger value between reported reservoir and modeled or reported lake volume. Column ‘Vol_src’ provides additional information regarding these distinctions.
Vol_res	Reported reservoir volume, or storage volume of added lake regulation, in million cubic meters (1 mcm = 0.001 km3). 0: no reservoir volume
Vol_src	1: ‘Vol_total’ is the reported total lake volume from literature 2: ‘Vol_total’ is the reported total reservoir volume from GRanD or literature 3: ‘Vol_total’ is the estimated total lake volume using the geostatistical modeling approach by Messager et al. (2016)
Depth_avg	Average lake depth, in meters. Average lake depth is defined as the ratio between total lake volume (‘Vol_total’) and lake area (‘Lake_area’).
Dis_avg	Average long-term discharge flowing through the lake, in cubic meters per second. This value is derived from modeled runoff and discharge estimates provided by the global hydrological model WaterGAP, downscaled to the 15 arc-second resolution of HydroSHEDS (see section 2.2 for more details) and is extracted at the location of the lake pour point. Note that these model estimates contain considerable uncertainty, in particular for very low flows. -9999: no data as lake pour point is not on HydroSHEDS landmask
Res_time	Average residence time of the lake water, in days. The average residence time is calculated as the ratio between total lake volume (‘Vol_total’) and average long-term discharge (‘Dis_avg’). Values below 0.1 are rounded up to 0.1 as shorter residence times seem implausible (and likely indicate model errors). -1: cannot be calculated as ‘Dis_avg’ is 0 -9999: no data as lake pour point is not on HydroSHEDS landmask
Elevation	Elevation of lake surface, in meters above sea level. This value was primarily derived from the EarthEnv-DEM90 digital elevation model at 90 m pixel resolution by calculating the majority pixel elevation found within the lake boundaries. To remove some artefacts inherent in this DEM for northern latitudes, all lake values that showed negative elevation for the area north of 60°N were substituted with results using the coarser GTOPO30 DEM of USGS at 1 km pixel resolution, which ensures land surfaces ≥0 in this region. Note that due to the remaining uncertainties in the EarthEnv-DEM90 some small negative values occur along the global ocean coastline south of 60°N which may or may not be correct.
Slope_100	Average slope within a 100 meter buffer around the lake polygon, in degrees. This value is derived from the EarthEnv-DEM90 digital elevation model at 90 m pixel resolution. Slopes for each pixel were computed with latitudinal corrections for the distortion in the XY spacing of geographic coordinates by approximating the geodesic distance between cell centers. For 12 lakes located above the northern limit of the EarthEnv-DEM90 digital elevation model (83°N), slopes were computed from the GTOPO30 DEM of USGS at 1 km pixel resolution. -1: slope values were not calculated for the largest lakes (polygon area ≥ 500 km2)
Wshd_area	Area of the watershed associated with the lake, in square kilometers. The watershed area is calculated by deriving and measuring the upstream contribution area to the lake pour point using the HydroSHEDS drainage network map at 15 arc-second resolution. -9999: no data as lake pour point is not on HydroSHEDS landmask
Pour_long	Longitude of the lake pour point, in decimal degrees.
Pour_lat	Latitude of the lake pour point, in decimal degrees.
"""

if True:
  task = f"""
  Your goal is to compute theoretical vs actual lake outlines and discuss how well they match. 
Pick at random a lake in an area that you expect to be moderately challenging.
Don't use an arid area.
For reference, use the Hydrolakes dataset '{hydrolakes_dataset_id}'.
Here are the feature properties for the Hydrolakes dataset:
""" + hydrolakes_schema

if False:
  task = """
Examine the datasets UCSB-CHG/CHIRPS/DAILY and NOAA/PERSIANN-CDR.
Your goal is to understand them and how they differ as deeply as possible.
Do not stop until your understanding is complete. Don't try to do everything at once.
You can return many short answers that help solve the problem - I will prompt you to continue.
"""

# Helper classes for IO and code execution

In [ ]:
# The agent needs three helper classes:
# * ColabIOManager knows how to interact with Colab
# * Supervisor keeps track of agent state and helps it terminate
# * CustomLoggingHandler captures logs from code parsing and execution

STARS = '*' * 20 + '\n'

class IOState(enum.StrEnum):
  THINKING = 'THINKING'
  RUNNING_CODE = 'RUNNING_CODE'
  WAITING_FOR_USER_INPUT = 'WAITING_FOR_USER_INPUT'
  DONE = 'DONE'


class IOManager:
  """Base class for I/O strategies."""

  def task_done(self, done_message: str='') -> None:
    if done_message.strip():
      self.display(f"Agent said: {done_message.strip()}\n")
    self.display("Task Done!")

  def display(self, text: str) -> None:
      raise NotImplementedError

  def set_state(self, state: IOState):
    raise NotImplementedError


class Supervisor:
  """Class responsible for controlling the agent."""

  # A public property indicating whether the agent is running
  running: bool
  _io_manager: IOManager

  def __init__(self, io_manager):
    self.running = False
    self._io_manager = io_manager

  def syscalls(self):
    return [self.task_done, get_dataset_description]

  def task_done(self, done_message: str='') -> None:
    """Signals the agent that the task is done to terminate execution."""
    self.running = False
    self._io_manager.task_done(done_message)

def set_cursor_waiting():
    js_code = """
    document.querySelector('body').style.cursor = 'wait';
    """
    display(HTML(f"<script>{js_code}</script>"))

def set_cursor_default():
    js_code = """
    document.querySelector('body').style.cursor = 'default';
    """
    display(HTML(f"<script>{js_code}</script>"))


class ColabIOManager(IOManager):
  """I/O Manager for Google Colab execution.

  This class exists to connect logging output from code parsing and execution
  to the Colab UI.
  """

  _ui_container: widgets.VBox

  def __init__(self):
    self._user_input_handler = None
    self._output_text = ''

    # Create output area with a unique ID
    self._output_id = f"output_{int(time.time())}"
    self._output_area = widgets.Output(
        layout=widgets.Layout(
            width='50%',
            height='600px',
            border='1px solid black',
            overflow='auto'
        )
    )
    # Add a unique CSS class to output area
    self._output_area.add_class(self._output_id)

    # Inner Container
    hbox = widgets.HBox([
        self._output_area,
        Map,
    ], layout=widgets.Layout(width='100%'))

    self._command_input = widgets.Text(
        placeholder='Type your message and press Enter...',
        description='❓',
        value = task,
        layout=widgets.Layout(width='95%')
    )

    # Container for the UI elements
    self._ui_container = widgets.VBox([
        hbox,
        self._command_input,
    ], layout=widgets.Layout(width='100%'))

    # Add CSS styling
    display(HTML("""
    <style>
    .widget-text input[type="text"] {
        width: 100% !important;
        padding: 8px;
        margin: 8px 0;
        box-sizing: border-box;
    }
    .jupyter-widgets-output-area {
        overflow-y: auto !important;
    }
    </style>
    """))

    self._command_input.on_submit(self._on_command)

  def start(self):
    display(self._ui_container)

  def _set_emoji(self, emoji):
    self._command_input.description = emoji

  def set_state(self, state: IOState):
    match state:
      case IOState.THINKING:
        self._set_emoji('🤔')
      case IOState.RUNNING_CODE:
        self._set_emoji('🌎')
      case IOState.WAITING_FOR_USER_INPUT:
        self._set_emoji('❓')
      case IOState.DONE:
        self._set_emoji('✅ ')
      case _:
        self._set_emoji('🦙')

  def _on_command(self, widget):
    """Accepts user input and passes it to the agent."""
    set_cursor_waiting()
    message = widget.value
    if message.strip():
      self.display(f"> {message}")
      widget.value = ''

      if self._user_input_handler:
        self._user_input_handler(message)
    set_cursor_default()

  def set_user_input_handler(self, handler):
    """Sets a handler function to be called when the user submits input."""
    self._user_input_handler = handler

  def display(self, text: str) -> None:
    text = text.strip()
    if text.endswith('None'):
      text = text[:-4]
    text = text.strip()
    self._output_text += (text + '\n')
    with self._output_area:
      display(HTML(f"<p style='white-space: pre-wrap;'>{text}</p>"))
    self._scroll_to_bottom()

  def _scroll_to_bottom(self):
    js_code = f"""
        requestAnimationFrame(() => {{
            const element = document.querySelector('.{self._output_id}');
            if (element) {{
                element.scrollTop = element.scrollHeight;
            }}
        }});
    """
    display(Javascript(js_code))


class CustomLoggingHandler(logging.Handler):
  """Csustom logging handler that sends agent internal logs to the Colab UI."""
  _io_manager: IOManager

  def __init__(self, io_manager):
    super().__init__(logging.INFO)
    self._io_manager = io_manager

  def emit(self, record):
    msg = self.format(record)
    self._io_manager.display(msg)

In [ ]:
io_manager = ColabIOManager()

# Colab agent

In [ ]:
import io
import tokenize

# TODO(simonf): move to functionsmith
def strip_comments(code):
  result = []
  tokens = tokenize.tokenize(io.BytesIO(code.encode()).readline)

  prev_toktype = tokenize.INDENT
  last_lineno = -1
  last_col = 0

  for tok in tokens:
    token_type = tok[0]
    token_string = tok[1]
    start_line, start_col = tok[2]
    end_line, end_col = tok[3]

    # The following two conditionals preserve indentation
    if start_line > last_lineno:
        last_col = 0
    if start_col > last_col:
        result.append(" " * (start_col - last_col))

    # Skip comments and docstrings
    if token_type == tokenize.COMMENT:
      pass
    elif token_type == tokenize.STRING:
      if prev_toktype != tokenize.INDENT:
        # This is likely a docstring; skip it
        if prev_toktype != tokenize.NEWLINE:
            # This is a string literal; keep it
            result.append(token_string)
    else:
      # This token is not a comment or docstring
      result.append(token_string)

    prev_toktype = token_type
    last_col = end_col
    last_lineno = end_line

  return ''.join(result)

In [ ]:
MAX_TURNS = 100

class Agent:
  """Main class for running the agent."""
  _llm: llm.LLM
  _num_turns: int

  # The twp dictionaries below will contain functions that the LLM can call.
  # The _syscalls dict has system functions - they are defined
  # by the agent beforehand. Their output is not intercepted.
  # The _functions dict will have functions dynamically created by the LLM.
  _syscalls: dict[str, code_parser.Function]
  _functions: dict[str, code_parser.Function]

  _io_manager: IOManager
  _supervisor: Supervisor
  _code_parser: code_parser.Parser
  _code_executor: executor.Executor

  def __init__(self, io_manager: IOManager, llm_interface: llm.LLM):
    self._io_manager = io_manager
    io_manager.set_user_input_handler(self.handle_user_input)
    self._supervisor = Supervisor(io_manager)

    self._llm = llm_interface
    self._io_manager.display(
        f'LLM: {llm_interface.__class__.__name__} {llm_interface._model_name}')
    self._syscalls = {}
    self._functions = {}

    logger = self._create_logger()
    self._code_parser = code_parser.Parser(logger)
    self._code_executor = executor.Executor(logger)

    self._extract_syscalls()

  def _create_logger(self):
    logger = logging.getLogger('EE Companion')
    logger.handlers = []
    logger.addHandler(CustomLoggingHandler(self._io_manager))
    logger.propagate = False
    return logger

  def _extract_syscalls(self):
    """Extracts system calls from the IO manager."""
    for method in self._supervisor.syscalls():
      supervisor_syscalls = self._code_parser.extract_functions(
          inspect.getsource(method))
      self._syscalls.update(supervisor_syscalls.functions)

  def _get_llm_response(self, question: str) -> code_parser.ParsedResponse:
    self._io_manager.set_state(IOState.THINKING)
    response = self._llm.chat(question)
    self._io_manager.display(f"Agent: {response}")
    return self._code_parser.extract_functions(response)

  def _handle_no_code_response(self):
    """Handles the case where the LLM response has no code."""
    self._io_manager.set_state(IOState.WAITING_FOR_USER_INPUT)
    self._supervisor.running = False

  def _execute_code(self, code: str) -> str:
    code_sans_comments = strip_comments(code)

    for existing in ['task_done', 'get_dataset_description']:
      if f'def {existing}(' in code_sans_comments:
        error = f"ERROR: DO NOT define {existing}; it's already defined."
        self._io_manager.display(error)
        return error

    if 'ee.Initialize(' in code_sans_comments:
      error = "ERROR: DO NOT call ee.Initialize. It was already called."
      self._io_manager.display(error)
      return error

    if 'geemap.Map(' in code_sans_comments:
      error = "ERROR: DO NOT create the geemap object. One is already created."
      self._io_manager.display(error)
      return error

    if 'getMapId' in code_sans_comments:
      error = "ERROR: DO NOT use getMapId."
      self._io_manager.display(error)
      return error

    sandbox_env = {
      'task_done': self._supervisor.task_done,
    }
    if 'task_done(' not in code_sans_comments:
      for layer in Map.layers[2:]:
        Map.remove(layer.name)

    self._io_manager.set_state(IOState.RUNNING_CODE)
    code_globals = {
        'ee': ee, 'geemap': geemap, 'Map': Map,
        'get_dataset_description': get_dataset_description
    }
    return self._code_executor.run_code(code, sandbox_env, code_globals)

  def handle_user_input(self, user_input):
    question = user_input + ' Call task_done() when you think the task is completed.'
    self._supervisor.running = True
    self._num_turns = 0

    # To respond to user input, we run an infinite loop until one of these
    # things happens:
    # 1. The agent returns a response without any code, which probably means
    #    it's asking the user something.
    # 2. The agent is no longer running, which probably means it thinks
    #    the task is done.
    while self._supervisor.running:
      self._io_manager.display(STARS)
      self._num_turns += 1
      if self._num_turns > MAX_TURNS:
        self._io_manager.display(f'REACHED {MAX_TURNS} TURNS, TERMINATING')
        return

      all_tools = copy.deepcopy(self._functions)
      all_tools.update(self._syscalls)
      function_definitions = '\n'.join([str(x) for x in all_tools.values()])
      question_with_tools = (
          f'{question}\n The following functions are available:\n'
          f'{function_definitions}')

      parsed_response = self._get_llm_response(question_with_tools)

      if not parsed_response.code and not parsed_response.functions:
        if parsed_response.error:
          # We couldn't parse the LLM-produced code, so we send the parsing
          # error to the LLM.
          question = parsed_response.error
          continue

        # The answer has no functions or top-level code.
        # We return control to the user.
        self._handle_no_code_response()
        return

      # If we are here, the response has top-level code, functions, or both.
      self._functions.update(parsed_response.functions)

      if not parsed_response.code:
        # The agent only defined functions, but gave no top-level code.
        question = 'go on'
        continue

      # Prepend all the code from previously defined functions
      previous_function_code = '\n'.join([x.code for x in self._functions.values()])
      code_with_tools =  previous_function_code + '\n' + parsed_response.code

      # The code execution output is saved into 'question', as it will be
      # sent to the LLM as the next user turn.
      question = self._execute_code(code_with_tools)
    # End of while loop

    self._io_manager.display(STARS)
    self._io_manager.set_state(IOState.DONE)


In [ ]:
ee_preamble = ("""
The client is running in a Python notebook with a geemap Map displayed.
The Map object already exists - do not reinitialize it.
If the code needs to add layers to the map, use the "Map.addLayer()" function
Note that every time you return code to be run, preexisting geemap layers are
deleted. ee.Authenticate() and ee.Initialize() have already been called.

When composing Python code, do not use getMapId. Do not escape quotation marks
and do not use line continuations in Python code.
Whatever the Python code prints will be returned as the execution result.
Each code invocation is separate, so local variables are not transferred
between them.

Make sure to use selfMask() right before vectorizing to shrink the mask to only
the valid area.

Be sure to use Python, not Javascript, syntax for keyword parameters in
Python code (that is, "function(arg=value)"). Use capitalized And for chaining
filters, as lowercase "and" is a reserved word in Python.

Before you choose a dataset, think about what kind of dataset would be most
suitable for the query. Also think about what zoom level would be suitable for
the query, keeping in mind that for high-resolution image collections higher
zoom levels are better to speed up tile loading. Earth Engine datasets often
can be very large - read them in small chunks. Do not aggregate properties over
the whole dataset.

Once you have chosen a dataset, read its description using the provided function
to see what spatial and temporal range it covers, what bands it has, as well as
to find the recommended visualization parameters. Explain
why you chose a specific dataset, zoom level and map location.

There is no ee.Date.now() function.

If you need to get a few pixel values, use ee.data.getPixels()

If you call functions that expect the Map object, pass them the real Map object
that is already defined and not a dummy one.

Before running a large-scale operation like mosaic() or reduceRegions(), first
try retrieving individual pixels
and then small data chunks using ee.data.computePixels()
with fileFormat=NUMPY_NDARRAY to verify that the source data for the area of
interest are present and have expected values. Example:
image_grid = {'dimensions': {'width': 5, 'height': 10}}
values = ee.data.computePixels({
        'expression': ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318'),
        'grid': image_grid,
        'bandIds': ['B1'],
        'fileFormat': 'NUMPY_NDARRAY'})

Prefer mosaicing image collections using functions like mosaic() or median(),
don't get individual images from collections via
'first()'. Choose a tile size and zoom level that will ensure the
tile has enough pixels in it to avoid graininess, but not so many
that processing becomes very expensive. Do not use wide date ranges
with collections that have many images, but remember that Landsat and
Sentinel-2 have revisit period of several days. Do not use sample
locations - try to come up with actual locations that are relevant to
the request.

If you are visualizing a layer on the map, you do not need to
clip it to the region of interest.

If you get the error "Parameter 'image' is required.", you are trying to get
an image from an empty image collection, likely because the collection filtering
is too strict.

When using first() to fetch an element from a FeatureCollection, wrap
the output in ee.Feature():
ee.Feature(ee.FeatureCollection("asset_id").filter()... .first())
Do this if you get the error "Parameter 'feature' is required." or if a layer
with a single feature is not added to geemap.

Use Landsat Collection 2, not Landsat Collection 1 ids. They look like this:
LANDSAT/LC08/C02/T1 or LANDSAT/LC08/C02/T1_L2

If you are getting
repeated errors when filtering by a time range, read the dataset description
to confirm that the dataset has data for the selected range.

If you need any clarification about Earth Engine functionality or additional
information to complete a task, please ask the user before proceeding.
If an Earth Engine operation might take a long time to process or return a
large amount of data, mention this and suggest ways to optimize or limit
the scope if necessary.
""")

# Start the agent

In [ ]:
system_instruction += ee_preamble
#llm_interface = llm.Gemini(system_instruction, api_key=userdata.get('GOOGLE_API_KEY'), model_name='gemini-2.0-flash')
#llm_interface = llm.Gemini(system_instruction, api_key=userdata.get('GOOGLE_API_KEY'), model_name='gemini-2.0-flash-thinking-exp-01-21')
llm_interface = llm.Gemini(system_instruction, api_key=userdata.get('GOOGLE_API_KEY'), model_name='gemini-2.5-pro-exp-05-06')
#llm_interface= llm.Claude(system_instruction, api_key=userdata.get('ANTHROPIC_API_KEY'), model_name='claude-3-7-sonnet-20250219')
#llm_interface = llm.ChatGPT(system_instruction, api_key=userdata.get('OPENAI_API_KEY'), model_name='gpt-4o')
#llm_interface = llm.DeepSeek(system_instruction, api_key=userdata.get('DEEPSEEK_API_KEY'))

agent = Agent(io_manager, llm_interface)
io_manager.start()

print("""
Legend:
❓ = Waiting for user input
🤔 = Thinking
🌎 = Running code
✅ = The task is done
""")
